# ホールドアウト検証

In [ ]:
import numpy as np
import pandas as pd
import librosa as lb
import os
import h5py
import joblib #並列処理のためのライブラリ
import shutil
from tqdm import tqdm # for文の進捗をプログレスバーで可視化する
import matplotlib.pyplot as plt
import glob
import wave

# モデルの再現性確保
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
np.random.seed(1)
tf.random.set_seed(1)

訓練データのパスを定義

In [ ]:
datas_dir = '../data/wav/'
'''
testdata : '210921_0032.wav' # 最終的な性能を評価するためのデータ
valdata : '210922_0033.wav' # 閾値などを検討する際に使用するデータ
'''
# testdata,valdataを除く8日分のデータを前半と後半に分ける
firsthalf_traindata = [datas_dir+'210919_0031.wav', datas_dir+'210923_0034.wav', datas_dir+'210924_0035.wav', datas_dir+'210925_0036.wav']
latterhalf_traindata = [datas_dir+'210926_0037.wav', datas_dir+'210927_0038.wav', datas_dir+'210928_0039.wav', datas_dir+'210929_0040.wav']

all_traindata = firsthalf_traindata+latterhalf_traindata # テストデータ,valdataを除く8日分のデータのパス
each_data_path_group = (firsthalf_traindata, latterhalf_traindata)

前半，後半それぞれのラベルデータ(異常ラベル追加前)の配列をタプルでまとめる

In [ ]:
label_data_dir = '../data/original_label/'

label_df_firsthalf_traindata = [] # 前半データの正解ラベルのデータフレームを格納したリスト
for i in firsthalf_traindata:
    label_data_name = i.split('/')[-1].replace('.wav', '.csv') 
    label_data_file = label_data_dir+label_data_name
    label_data_df=pd.read_csv(label_data_file, skiprows=4, usecols=[1], header=None) 
    label_array = label_data_df.iloc[:, 0].values
    label_df_firsthalf_traindata.append(label_array)

label_df_latterhalf_traindata = [] # 後半データの正解ラベルのデータフレームを格納したリスト
for i in latterhalf_traindata:
    label_data_name = i.split('/')[-1].replace('.wav', '.csv') 
    label_data_file = label_data_dir+label_data_name
    label_data_df=pd.read_csv(label_data_file, skiprows=4, usecols=[1], header=None)
    label_array = label_data_df.iloc[:, 0].values
    label_df_latterhalf_traindata.append(label_array)
    
each_data_original_array_group = (label_df_firsthalf_traindata, label_df_latterhalf_traindata)

閾値を求める際に使用するデータのパス

In [ ]:
threshold_data_path = '../data/threshold/NormalandAnormSound.hdf5'

各日のメルスペクトログラムが保存されたhdfファイルのパス

In [ ]:
eachday_hdf = '../data/eachday/eachday.hdf5'

メルスペクトログラムのパラメータ

In [ ]:
sr = 16000 # サンプリング周波数
duration = 1 # メルスペクトログラムの時間幅[s]
melparams = {'sr':sr, 'n_mels':128, 'fmin':0, 'fmax':sr/2} # メルスペクトログラムのパラメータ

データ拡張用のオブジェクト

In [ ]:
# データ拡張のためのクラスをimport 
from audio_DA import Multiple, ResampleWaveform, GaussianNoiseSNR, PinkNoiseSNR, PitchShift, TimeShift, VolumeShift

# 複数の音声処理の関数
# なんの音声処理をするかを決める
transform = Multiple([
    GaussianNoiseSNR(min_snr=15, max_snr=30),
    PinkNoiseSNR(min_snr=8, max_snr=30),
    PitchShift(max_steps=2, sr=sr),
    TimeShift(sr=sr),
    VolumeShift(mode="cosine")
])

In [ ]:
from AutoEncoder import AutoEncoder

SemiSLAutoEncoder = AutoEncoder(
    each_data_path_group=each_data_path_group, each_data_original_array_group=each_data_original_array_group, 
    threshold_data_path=threshold_data_path, eachday_hdf=eachday_hdf, transform=transform, 
    sr=sr, duration=duration, melparams=melparams)

## 学習フェーズ

In [ ]:
SemiSLAutoEncoder.co_fit(2, 2048, 60)

In [ ]:
SemiSLAutoEncoder.all_fit(2048, 100)

## 推論フェーズ

In [ ]:
# duration単位の異常度の結果を0.2秒単位に変換する
def result_to_200ms(result, duration):
    import numpy as np
    total_time = len(result)*duration
    step =int(duration/0.2)
    result_200ms = np.zeros(int(total_time/0.2))
    for i in range(len(result)):
        if i<len(result)-1:
            result_200ms[i*step:(i+1)*step] = result[i]
        else:
            result_200ms[i*step:] = result[i]

    return result_200ms

'''
0.２秒単位の配列を1秒単位の配列に変換する
'''
def label_to_sec(label):
    import numpy as np
    unit_sec = int(1/0.2)
#     print(type(unit_sec))
#     print(unit_sec)
    label_sec_array = label[unit_sec::unit_sec]

    return label_sec_array

In [ ]:
# 適合率・再現率・F値の可視化
def fig_pre_re_f(list, pre,re,fscore):
    fig, ax = plt.subplots(facecolor="w")
    ax.set_xlabel("Threshold")
    ax.grid()

    ax.plot(list, pre, label="Precision", marker='o')
    ax.plot(list, re, label="Recall", marker='s')
    ax.plot(list, fscore, label="fscore", marker='v')
    ax.legend()
# F値の可視化
def fig_fscore(list,fscores):
    fig, ax = plt.subplots(facecolor="w")
    ax.set_xlabel("Threshold")
    ax.grid()
    ax.plot(list, fscores, label="fscore")
    ax.legend()
# 適合率の可視化
def fig_precision(list,pre):
    fig, ax = plt.subplots(facecolor="w")
    ax.set_xlabel("Threshold")
    ax.grid()
    ax.plot(list, pre, label="Precision")
    ax.legend()
# 再現率の可視化
def fig_recall(list,re):
    fig, ax = plt.subplots(facecolor="w")
    ax.set_xlabel("Threshold")
    ax.grid()
    ax.plot(list, re, label="Recall")
    ax.legend()
    
# PR曲線
def fig_PR(test, score, bins):
    """
    test:一列目に正解ラベルが記入されたデータフレーム
    score：異常度
    bins：PR曲線の閾値の数(計算時に切り捨てをしているので正確にこの数にはならない)
    """
    from sklearn.metrics import precision_recall_curve, auc
    
    test_array = test.iloc[:, 0].values
    print('test_array:', test_array)
    print('test:', test)
    print(np.bincount(test_array))  
    test_array = np.where(test_array==2, 1, test_array)
    print(np.bincount(test_array))   
    
    _, _, thresholds = precision_recall_curve(test_array, score)
    interval = int(len(thresholds)/bins)
    thr = thresholds[:-1][::interval]
    print(thresholds[-1])
    thr =  np.append(thr, thresholds[-1])
    
    precision = np.zeros(len(thr))
    recall = np.zeros(len(thr))
    for i in range(len(thr)):
        precision[i], recall[i]=validate(test, score, thr[i])
    auc_score = auc(recall, precision)
    print(f'auc:{auc_score}')
        
    # PR曲線
    fig, ax = plt.subplots(facecolor="w", figsize=(5, 5))
    plt.gca().yaxis.set_major_formatter(plt.FormatStrFormatter('%.2f')) # 軸メモリの桁数
    plt.gca().xaxis.set_major_formatter(plt.FormatStrFormatter('%.2f')) #軸メモリの桁数
    ax.grid()
    ax.plot(recall, precision)
    ax.set_xlabel('再現率', fontname="MS Mincho", fontsize=15)
    ax.set_ylabel('適合率', fontname="MS Mincho", fontsize=15)
    
    return auc_score

ハイパーパラメータの調整

In [ ]:
from validate import validate
import matplotlib.pyplot as plt

'''
testdata : '210921_0032.wav' # 最終的な性能を評価するためのデータ
valdata : '210922_0033.wav' # 閾値などを検討する際に使用するデータ
'''

# 検証データ
testdata = '210922_0033'
testdata_path = '../data/wav/'+testdata+'.wav'
label_data_file = '../data/fixed_label/-2109230700.csv'

model_path = '../trainedmodel/alltraindata/sslautoencoder.hd5'

(差分画像を何倍するか)

In [ ]:
list = [0.3, 0.29, 0.28, 0.27, 0.26, 0.25, 0.24, 0.23, 0.22, 0.21, 0.2]

pre=[]
re=[]
for i in list:
    print('='*50)
    print(f'パラメータ：{i}')
    anomary_scores, notburied_anomary_scores, abnorm_thr = SemiSLAutoEncoder.predict(testdata_path, model_path, xbox=1.5, xdiff=i)
    anomary_scores_200ms = result_to_200ms(anomary_scores, duration) #duration:フレームの時間幅
    notburied_anomary_scores_200ms = result_to_200ms(notburied_anomary_scores, duration) #duration:フレームの時間幅    
    testlabel_data_df=pd.read_csv(label_data_file, skiprows=4, usecols=[0, 1], header=None)  # fileをデータフレームに出力
    testlabel_data_array = testlabel_data_df.iloc[:, 1].values    
    if len(testlabel_data_array)<=len(anomary_scores_200ms): 
        anomary_scores_200ms = anomary_scores_200ms[:len(testlabel_data_array)]
        notburied_anomary_scores_200ms = notburied_anomary_scores_200ms[:len(testlabel_data_array)]
    else:
        testlabel_data_array = testlabel_data_array[:len(anomary_scores_200ms)]

    df = pd.DataFrame(data=testlabel_data_array, columns=['label'])
    
    print('差分画像を使った異常音判別結果')
    pre_score, re_score=validate(df[:198000], notburied_anomary_scores_200ms[:198000], abnorm_thr)#ノイズが少ない部分（11時間分)のデータを使う
    pre.append(pre_score)
    re.append(re_score)

In [ ]:
# F値
fscores = []
for i in range(len(pre)):
    f = (2*pre[i]*re[i])/(pre[i]+re[i])
    fscores.append(f)

In [ ]:
fig_pre_re_f(list, pre,re,fscores)
fig_fscore(list,fscores)
fig_precision(list, pre)
fig_recall(list, re)

テストデータに対する異常度算出

In [ ]:
'''
testdata : '210921_0032.wav' # 最終的な性能を評価するためのデータ
valdata : '210922_0033.wav' # 閾値などを検討する際に使用するデータ
'''
# テストデータ
testdata = '210921_0032'
testdata_path = '../data/wav/'+testdata+'.wav'
label_data_file = '../data/fixed_label/-2109220600.csv'

model_path = '../trainedmodel/alltraindata/sslautoencoder.hd5'

In [ ]:
anomary_scores, notburied_anomary_scores, abnorm_thr = SemiSLAutoEncoder.predict(testdata_path, model_path, xbox=1.5, xdiff=0.26)

In [ ]:
anomary_scores_200ms = result_to_200ms(anomary_scores, duration) 
notburied_anomary_scores_200ms = result_to_200ms(notburied_anomary_scores, duration) 

テストデータの正解ラベルを読み込む

In [ ]:
testlabel_data_df=pd.read_csv(label_data_file, skiprows=4, usecols=[0, 1], header=None)  # fileをデータフレームに出力
display(testlabel_data_df)
testlabel_data_array = testlabel_data_df.iloc[:, 1].values

In [ ]:
# 騒音データと音声データの長さが違うので，正解ラベルの配列と異常度の配列の長さをそろえる必要がある
if len(testlabel_data_array)<=len(anomary_scores_200ms): 
    anomary_scores_200ms = anomary_scores_200ms[:len(testlabel_data_array)]
    notburied_anomary_scores_200ms = notburied_anomary_scores_200ms[:len(testlabel_data_array)]
else:
    testlabel_data_array = testlabel_data_array[:len(anomary_scores_200ms)]

評価値算出

In [ ]:
from validate import validate

df = pd.DataFrame(data=testlabel_data_array, columns=['label'])
auc = fig_PR(df, anomary_scores_200ms, 50)

In [ ]:
df = pd.DataFrame(data=testlabel_data_array, columns=['label'])
notburied_auc = fig_PR(df, notburied_anomary_scores_200ms, 50)

In [ ]:
df = pd.DataFrame(data=testlabel_data_array, columns=['label'])
print('差分画像を使わない異常音判別結果')
validate(df, anomary_scores_200ms, abnorm_thr)
print(f'AUC:{auc}')

print('差分画像を使った異常音判別結果')
validate(df, notburied_anomary_scores_200ms, abnorm_thr)
print(f'AUC:{notburied_auc}')

グラフ描画

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

# 0.2秒単位の配列を1秒単位の正解ラベルに変換する
anomary_scores_sec_array = label_to_sec(anomary_scores_200ms)
notburied_anomary_scores_sec_array = label_to_sec(notburied_anomary_scores_200ms)
label_sec_array = label_to_sec(testlabel_data_array)

time_index = pd.date_range(start='2022-09-21 06:00:00', end='2022-09-21 22:00:00', freq='S')
time_index = time_index[:len(anomary_scores_sec_array)]

graphnum = -(-len(time_index)//3600)
# plt.rcParams['font.size']=18
figure, axes = plt.subplots(graphnum, 1, figsize=(35, 300))
for i in range(graphnum):

    axes[i].plot(date2num(time_index), (notburied_anomary_scores_sec_array), label="異常度")
    axes[i].fill_between(date2num(time_index), label_sec_array, color='red', alpha=0.5, label="異常ラベル")
    
    axes[i].set_xlabel('時刻', fontname="MS Mincho", fontsize=40)
    axes[i].set_ylabel('異常度', fontname="MS Mincho", fontsize=40)
    axes[i].xaxis.set_tick_params(labelsize = 35)
    axes[i].yaxis.set_tick_params(labelsize = 35)

    axes[i].legend(loc='lower center', bbox_to_anchor=(.5, 1.), ncol=2, prop={'family':"MS Mincho", 'size':35})

#     axes[i].set_xlabel('Time')
#     axes[i].set_ylabel('Abnormality')

    axes[i].set_ylim(abnorm_thr, 0.003)
    if i<(graphnum-1):
        axes[i].set_xlim(
            date2num(time_index[3600*i]), date2num(time_index[3600*(i+1)]))
    else:
        axes[i].set_xlim(date2num(time_index[3600*i]), date2num(time_index[-1]))
        
    axes[i].xaxis.set_major_formatter(DateFormatter('%H:%M'))
    #5分おきにラベル
    Minute1=mdates.MinuteLocator(range(60),5)   
    axes[i].xaxis.set_major_locator(Minute1)